# Collinear Points

### Collinear points

Definition of collinearity[1]: In geometry, collinearity of a set of points is the property of their lying on a single line.

![](non-collinear-points.jpg)

### Description of the Approach

The goal is to find sets of collinear points from a list of points:

1. List all pairs of points. You can do that efficiently in spark by computing cartesian product of the list of points with itself. For example, given three points $[(1,0), (2,0), (3,0)]$, we construct a list of nine pairs  
$[((1,0),(1,0)),((1,0), (2,0)),((1,0),(3,0))$  
$((2,0),(1,0)),((2,0), (2,0)),((2,0),(3,0))$  
$((3,0),(1,0)),((3,0), (2,0)),((3,0),(3,0))]$  

2. Remove the pairs in which the same point appears twice such as $((2,0),(2,0))$. After these elimination you end up (for this example) with a list of just six pairs:  
$[((1,0),(2,0)),((1,0),(3,0)),((2,0),(1,0)),((2,0),(3,0)),((3,0),(1,0)),((3,0),(2,0))]$

2. For each pair of points, find the parameterization $(a,b)$ of the line connecting them as described above.

3. Group the pairs according to their parameters. Clearly, if two pairs have the same $(a,b)$ values, all points in the two pairs lie on the same line.

3. Eliminate the groups that contain only one pair (any pair of points defines a line).
4. In each of the remaining groups, unpack the point-pairs to identify the individual points.
Note that if a set of points $(x_1,y_1),\ldots,(x_k,y_k)$ lie on the same line then each point will appear $k-1$ times in the list of point-pairs. You therefore need to transform the list of points into sets to remove duplicates.

5. Output the sets of 3 or more colinear points.


In [4]:
#Spark setup
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("Collinear Points").setMaster("local[4]") #Initialize spark context using 4 local cores as workers
sc = SparkContext(conf=conf)    

from pyspark.rdd import RDD

### Helper Functions

The function <font color="blue">format_result</font> takes an element of the form shown below in the example. It outputs a tuple of all points that are collinear (shown below).
Input: ((A,slope), [C1,..., Ck]) where each of A, C1, ..., Ck is a point of form (Ax, Ay) and slope is of type float.

**<font color="magenta" size=2>Example Code</font>**
``` python
my_input = (((2, 1), 0.5), [(4, 2), (6, 3)]) 
format_result(my_input)
```
Output: (C1,..., Ck, A) each of A,C1,...,Ck is a point of form (Ax, Ay)

The function <font color="blue">non_duplicates</font> checks if a set of points contains duplicates or not.
Input: Pair (A,B) where A and B are of form (Ax, Ay) and (Bx, By) respectively.

The function <font color="blue">get_cartesian</font> does a cartesian product of an RDD with itself and returns an RDD with <b>DISTINCT</b> pairs of points.


The function <font color="blue">find_slope</font> computes slope between points A and B and returns it in the format below:

Input : Pair (A,B) where A and B are of form (Ax, Ay) and (Bx, By) respectively. 
Output: Pair ((A,slope), B) where A and B have the same definition as input and slope refers to the slope of the line segment           connecting point A and B.

### Functions

In [11]:
def format_result(x):
    x[1].append(x[0][0])
    return tuple(x[1])

def to_sorted_points(x):
    return tuple(sorted(x))

def to_tuple(x):
    #if x is string ("1 2")
    a, b = x.split(" ")
    return (int(a), int(b))

def non_duplicates(x):
    if(x[0][0] == x[1][0] and x[0][1] == x[1][1]):
        return False
    else:
        return True

def get_cartesian(rdd):
    rddl = rdd.cartesian(rdd).filter(non_duplicates)
    return rddl

def find_slope(x):
    if(x[1][0] != x[0][0]):
        a = (x[1][1] - x[0][1])/(x[1][0] - x[0][0])
        return ((x[0], a), x[1])
    else:
        return ((x[0],'inf'), x[1])

### Find_collinear



The function <font color="blue">find_collinear</font> finds the set of collinear points.

Input: An RDD (which is the output of the get_cartesian() function. 

Output: An RDD containing the list of collinear points formatted according to the <font color="blue">format_result</font> function.

Approach:
1. Find the slope of the line between all pairs of points A = (Ax, Ay) and B = (Bx, By).
2. For each (A, B), find all points C = ((C1x, C1y), (C2x, C2y), ... (Cnx, Cny)) 
   where slope of (A,B) = slope of (A, Ci).
3. Return (A, B, Ck) where Ck = all points of C which satisfy the condition.

In [3]:
def find_collinear(rdd):
    rddt = rdd.map(lambda p: find_slope(p)).groupByKey().mapValues(list).map(format_result).map(to_sorted_points).filter(lambda t: len(t) > 2).distinct()
    return rddt

#### Unit Tests

In [5]:
def verify_collinear_sets(collinearpointsRDD, testlist):
    collinearpoints = [tuple(sorted(x)) for x in list(set(collinearpointsRDD.collect()))]
    testlist = [tuple(sorted(x)) for x in list(set(testlist))]
    return set(collinearpoints) == set(testlist)

In [6]:
test_rdd = sc.parallelize([((4, 2), (2, 1)), ((4, 2), (-3, 4)), ((4, 2), (6, 3)), ((2, 1), (4, 2)), ((2, 1), (-3, 4)), ((2, 1), (6, 3)), ((-3, 4), (4, 2)), ((-3, 4), (2, 1)), ((-3, 4), (6, 3)), ((6, 3), (4, 2)), ((6, 3), (2, 1)), ((6, 3), (-3, 4))])
assert isinstance(find_collinear(test_rdd), RDD) == True, "Incorrect return type"

In [7]:
assert verify_collinear_sets(find_collinear(test_rdd), [((2, 1), (4, 2), (6, 3))]), "ERROR"

## Process to get a set of  collinear points

In [12]:
def build(rdd):
    rdd = rdd.map(lambda s: to_tuple(s))
    rdd = get_cartesian(rdd)
    rdd = find_collinear(rdd)
    rdd = rdd.map(to_sorted_points)
    return rdd

In [15]:
def process(file):
    rdd = sc.textFile(file)
    rdd = build(rdd)
    res = set(rdd.collect())    
    return res

In [16]:
assert process("data.txt") == {((-2, -2), (1, 1), (2, 2), (3, 3)), ((0, 1), (3, 4), (5, 6)), ((0, -3), (0, 1), (0, 5))}, "Your implementation of build_collinear_set is not correct."